In [3]:
import math
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from tensorboardX import SummaryWriter
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm, trange
import pandas as pd
import time
import mpmath
import os

import VMF

GPUs are used!
Classes loaded
0
dict_items([('l0', BayesianLinearLast()), ('l1', BayesianLinear(
  (weight): vMF()
)), ('l2', BayesianLinearLast())])


TypeError: linear(): argument 'input' (position 1) must be Tensor, not int

In [ ]:
#epochs = 225
#trtimes  = np.zeros(epochs)
# make inference on 10 networks
#for i in range(0, 1):
#    print(i)
#    torch.manual_seed(i)
#    net = VMF.BayesianNetwork().to(VMF.DEVICE)
#    optimizer = optim.Adam(net.parameters(), lr=0.0001)
#    for epoch in range(epochs):
#
#        trtimes[epoch] = VMF.train(net, optimizer, epoch, i)
#        print(net.l1.weight_mu.mean())
#
#    res = VMF.test_ensemble()
#
#    np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")
layers = [(256, 400),(400, 600),(400, 600),(600, 5)]
self_layerdict = {}
i = 0
while i<len(layers):
            name = 'self_l'+str(i)
            self_layerdict[name] = layers[i]
            i = i+1
            print(name)
for k,v in self_layerdict.items():
    exec("%s = %s" % (k, v))
self_l1